Package Setup

Links: 

https://www.datagraphi.com/blog/post/2019/12/17/how-to-find-the-optimum-number-of-hidden-layers-and-nodes-in-a-neural-network-model

https://www.tidyverse.org/blog/2020/02/slider-0-1-0/

In [ ]:
import datetime
from packaging import version
from collections import Counter
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from sklearn.metrics import mean_squared_error as MSE

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
%matplotlib inline
np.set_printoptions(precision=3, suppress=True)

Load Data


In [ ]:
training = pd.read_csv("https://raw.githubusercontent.com/nateschor/Fantasy-Forecasting/main/data/intermediate/all_data_training_normalized_1871_to_2010.csv")
validation = pd.read_csv("https://raw.githubusercontent.com/nateschor/Fantasy-Forecasting/main/data/intermediate/all_datavalidation_normalized_2011_to_2016.csv")
test = pd.read_csv("https://raw.githubusercontent.com/nateschor/Fantasy-Forecasting/main/data/intermediate/all_datatest_normalized_2017_to_2019.csv")

training["split"] = "Training"
validation["split"] = "Validation"
test["split"] = "Test"

In [ ]:
training

,playerID,nameFirst,nameLast,yearID,teamID,Era,G,AB,Hits,Runs_Scored,Singles,Doubles,Triples,HR,TB,RBI,BB,K,HBP,Sacrifices,SB,CS,Points,birthMonth,birthCountry,birthState,birthCity,bats,throws,season_number,Points_last_season,Points_2_season,Points_3_season,birthYear,weight,height,Age,split
0,aardsda01,David,Aardsma,2004,SFN,Long Ball,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,Dec,USA,CO,Denver,R,R,1,0.0,0.0,0.0,0.943038,0.588235,0.800,0.162791,Training
1,aardsda01,David,Aardsma,2006,CHN,Long Ball,45,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.5,Dec,USA,CO,Denver,R,R,2,0.0,0.0,0.0,0.943038,0.588235,0.800,0.209302,Training
2,aardsda01,David,Aardsma,2007,CHA,Long Ball,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,Dec,USA,CO,Denver,R,R,3,0.5,0.0,0.0,0.943038,0.588235,0.800,0.232558,Training
3,aardsda01,David,Aardsma,2008,BOS,Long Ball,47,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1.0,Dec,USA,CO,Denver,R,R,4,0.0,0.5,0.0,0.943038,0.588235,0.800,0.255814,Training
4,aardsda01,David,Aardsma,2009,SEA,Long Ball,73,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,Dec,USA,CO,Denver,R,R,5,-1.0,0.0,0.5,0.943038,0.588235,0.800,0.279070,Training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88916,zuverge01,George,Zuverink,1959,BAL,Integration,6,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,3.0,Aug,USA,MI,Holland,R,R,10,9.5,6.0,3.5,0.582278,0.509804,0.825,0.441860,Training
88917,zwilldu01,Dutch,Zwilling,1910,CHA,Dead Ball,27,87,16,7,11,5,0,0,21,5,11,16,1,0,1,0,0.0,Nov,USA,MO,St. Louis,L,L,1,0.0,0.0,0.0,0.354430,0.372549,0.575,0.139535,Training
88918,zwilldu01,Dutch,Zwilling,1914,CHF,Dead Ball,154,592,185,91,123,38,8,16,287,95,46,68,1,0,21,0,0.0,Nov,USA,MO,St. Louis,L,L,2,0.0,0.0,0.0,0.354430,0.372549,0.575,0.232558,Training
88919,zwilldu01,Dutch,Zwilling,1915,CHF,Dead Ball,150,548,157,65,105,32,7,13,242,94,67,65,2,0,24,0,0.0,Nov,USA,MO,St. Louis,L,L,3,0.0,0.0,0.0,0.354430,0.372549,0.575,0.255814,Training


In [ ]:
np.min(training["yearID"])

1871

In [ ]:
validation

,playerID,nameFirst,nameLast,yearID,teamID,Era,G,AB,Hits,Runs_Scored,Singles,Doubles,Triples,HR,TB,RBI,BB,K,HBP,Sacrifices,SB,CS,Points,birthMonth,birthCountry,birthState,birthCity,bats,throws,season_number,Points_last_season,Points_2_season,Points_3_season,birthYear,weight,height,Age,split
0,aardsda01,David,Aardsma,2012,NYA,Long Ball,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,Dec,USA,CO,Denver,R,R,7,0.0,0.0,-1.0,0.943038,0.588235,0.800,0.348837,Validation
1,aardsda01,David,Aardsma,2013,NYN,Long Ball,43,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,Dec,USA,CO,Denver,R,R,8,0.0,0.0,0.0,0.943038,0.588235,0.800,0.372093,Validation
2,aardsda01,David,Aardsma,2015,ATL,Long Ball,33,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1.0,Dec,USA,CO,Denver,R,R,9,0.0,0.0,0.0,0.943038,0.588235,0.800,0.418605,Validation
3,abadfe01,Fernando,Abad,2011,HOU,Long Ball,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,Dec,D.R.,La Romana,La Romana,L,L,2,-1.0,0.0,0.0,0.968354,0.666667,0.775,0.232558,Validation
4,abadfe01,Fernando,Abad,2012,HOU,Long Ball,37,7,1,0,1,0,0,0,1,0,0,3,0,0,0,0,-0.5,Dec,D.R.,La Romana,La Romana,L,L,3,0.0,-1.0,0.0,0.968354,0.666667,0.775,0.255814,Validation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8600,zuninmi01,Mike,Zunino,2014,SEA,Long Ball,131,438,87,51,43,20,2,22,177,60,17,158,17,4,0,3,347.0,Mar,USA,FL,Cape Coral,R,R,2,139.0,0.0,0.0,1.006329,0.666667,0.775,0.162791,Validation
8601,zuninmi01,Mike,Zunino,2015,SEA,Long Ball,112,350,61,28,39,11,0,11,105,28,21,132,5,10,0,1,183.0,Mar,USA,FL,Cape Coral,R,R,3,347.0,139.0,0.0,1.006329,0.666667,0.775,0.186047,Validation
8602,zuninmi01,Mike,Zunino,2016,SEA,Long Ball,55,164,34,16,15,7,0,12,77,31,21,65,6,1,0,0,169.5,Mar,USA,FL,Cape Coral,R,R,4,183.0,347.0,139.0,1.006329,0.666667,0.775,0.209302,Validation
8603,zychto01,Tony,Zych,2015,SEA,Long Ball,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,Aug,USA,IL,Monee,R,R,1,0.0,0.0,0.0,1.000000,0.490196,0.800,0.209302,Validation


In [ ]:
test

,playerID,nameFirst,nameLast,yearID,teamID,Era,G,AB,Hits,Runs_Scored,Singles,Doubles,Triples,HR,TB,RBI,BB,K,HBP,Sacrifices,SB,CS,Points,birthMonth,birthCountry,birthState,birthCity,bats,throws,season_number,Points_last_season,Points_2_season,Points_3_season,birthYear,weight,height,Age,split
0,abadfe01,Fernando,Abad,2017,BOS,Long Ball,48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,Dec,D.R.,La Romana,La Romana,L,L,9,-1.0,-1.0,0.0,0.968354,0.666667,0.775,0.372093,Test
1,abadfe01,Fernando,Abad,2019,SFN,Long Ball,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,Dec,D.R.,La Romana,La Romana,L,L,10,0.0,-1.0,-1.0,0.968354,0.666667,0.775,0.418605,Test
2,abreubr01,Bryan,Abreu,2019,HOU,Long Ball,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,Apr,D.R.,Distrito Nacional,Santo Domingo,R,R,1,0.0,0.0,0.0,1.044304,0.627451,0.750,0.139535,Test
3,abreujo02,Jose,Abreu,2017,CHA,Long Ball,156,621,189,95,107,43,6,33,343,102,35,119,15,4,3,0,857.0,Jan,Cuba,Cienfuegos,Cienfuegos,R,R,4,751.0,763.0,811.0,0.981013,0.725490,0.800,0.325581,Test
4,abreujo02,Jose,Abreu,2018,CHA,Long Ball,128,499,132,68,73,36,1,22,236,78,37,109,11,6,2,0,596.5,Jan,Cuba,Cienfuegos,Cienfuegos,R,R,5,857.0,751.0,763.0,0.981013,0.725490,0.800,0.348837,Test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4589,zobribe01,Ben,Zobrist,2019,CHN,Long Ball,47,150,39,24,33,5,0,1,47,17,23,24,1,2,0,0,163.0,May,USA,IL,Eureka,B,R,15,594.5,470.0,739.5,0.943038,0.568627,0.800,0.511628,Test
4590,zuninmi01,Mike,Zunino,2017,SEA,Long Ball,124,387,97,52,47,25,0,25,197,64,39,160,8,1,1,0,417.5,Mar,USA,FL,Cape Coral,R,R,5,169.5,183.0,347.0,1.006329,0.666667,0.775,0.232558,Test
4591,zuninmi01,Mike,Zunino,2018,SEA,Long Ball,113,373,75,37,37,18,0,20,153,44,24,150,6,2,0,0,278.5,Mar,USA,FL,Cape Coral,R,R,6,417.5,169.5,183.0,1.006329,0.666667,0.775,0.255814,Test
4592,zuninmi01,Mike,Zunino,2019,TBA,Long Ball,90,266,44,30,24,10,1,9,83,32,20,98,3,0,0,0,165.5,Mar,USA,FL,Cape Coral,R,R,7,278.5,417.5,169.5,1.006329,0.666667,0.775,0.279070,Test


One Hot Encoding

In [ ]:
all_data = pd.concat([training, validation, test])

char_categorical_vars = ["birthCountry", "birthState", "birthCity", "bats", "throws", "split"] # "playerID"

In [ ]:
all_dummies = pd.get_dummies(all_data[char_categorical_vars])

In [ ]:
def Perform_One_Hot_Encoding(dataset):

  char_dummies = pd.get_dummies(dataset[char_categorical_vars])
  birthMonth_dummies = pd.get_dummies(dataset["birthMonth"])
  season_number_dummies = pd.get_dummies(dataset["season_number"])

  return pd.concat([dataset.Points, dataset.Points_last_season, dataset.Points_2_season, dataset.Points_3_season, dataset.birthYear, dataset.weight, dataset.height, dataset.Age, char_dummies, birthMonth_dummies, season_number_dummies], axis=1)




In [ ]:
all_ready = Perform_One_Hot_Encoding(all_data)

training_ready = all_ready[all_ready["split_Training"] == 1]
validation_ready = all_ready[all_ready["split_Validation"] == 1]
test_ready = all_ready[all_ready["split_Test"] == 1]

In [ ]:
training_ready

,Points,Points_last_season,Points_2_season,Points_3_season,birthYear,weight,height,Age,birthCountry_Aruba,birthCountry_Australia,birthCountry_Bahamas,birthCountry_Belgium,birthCountry_Brazil,birthCountry_CAN,birthCountry_Colombia,birthCountry_Cuba,birthCountry_Curacao,birthCountry_Czech Republic,birthCountry_D.R.,birthCountry_France,birthCountry_Germany,birthCountry_Greece,birthCountry_Guam,birthCountry_Honduras,birthCountry_Hong Kong,birthCountry_Ireland,birthCountry_Italy,birthCountry_Jamaica,birthCountry_Japan,birthCountry_Latvia,birthCountry_Lithuania,birthCountry_Mexico,birthCountry_Netherlands,birthCountry_Nicaragua,birthCountry_P.R.,birthCountry_Panama,birthCountry_Peru,birthCountry_Poland,birthCountry_Russia,birthCountry_Saudi Arabia,...,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.0,0.0,0.943038,0.588235,0.800,0.162791,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.5,0.0,0.0,0.0,0.943038,0.588235,0.800,0.209302,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,0.5,0.0,0.0,0.943038,0.588235,0.800,0.232558,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-1.0,0.0,0.5,0.0,0.943038,0.588235,0.800,0.255814,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.0,-1.0,0.0,0.5,0.943038,0.588235,0.800,0.279070,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88916,3.0,9.5,6.0,3.5,0.582278,0.509804,0.825,0.441860,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
88917,0.0,0.0,0.0,0.0,0.354430,0.372549,0.575,0.139535,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
88918,0.0,0.0,0.0,0.0,0.354430,0.372549,0.575,0.232558,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
88919,0.0,0.0,0.0,0.0,0.354430,0.372549,0.575,0.255814,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Make sure no observations dropped

In [ ]:
assert training_ready.shape[0] == training.shape[0]
assert validation_ready.shape[0] == validation.shape[0]
assert test_ready.shape[0] == test.shape[0]

Make sure training, validation and test have same columns

In [ ]:
assert training_ready.shape[1] == validation_ready.shape[1]
assert validation_ready.shape[1] == test_ready.shape[1]
assert test_ready.shape[1] == training_ready.shape[1]

In [ ]:
training_x = training_ready.drop("Points", axis = 1)
training_y = training_ready["Points"]
val_x = validation_ready.drop("Points", axis = 1)
val_y = validation_ready["Points"]
test_x = test_ready.drop("Points", axis = 1)
test_y = test_ready["Points"]

In [ ]:
def Create_Num_Layers_And_Nodes(number_hidden_layers, number_input_nodes = training_x.shape[1], number_output_nodes = 1):

  nodes_increment = np.geomspace(number_input_nodes, number_output_nodes, num = number_hidden_layers, dtype='int')
  nodes_increment[0] = training_x.shape[1]
  nodes_increment[-1] = 1
  return nodes_increment.tolist()

  # return np.floor(nodes_increment)

In [ ]:
def Run_Model(hidden_layers_list):

  hidden_layers_list = hidden_layers_list[0:len(hidden_layers_list) - 1]

  model = models.Sequential()
  for i in range(1, len(hidden_layers_list)):

    if i == 1:
      model.add(layers.Dense(hidden_layers_list[1], activation='selu',
      input_shape=(training_x.shape[1],)))

    else: 
      model.add(layers.Dense(hidden_layers_list[i], activation='selu'))
      tf.keras.layers.Dropout(0.3)

  model.add(layers.Dense(1)) 
  return model

In [ ]:
nodes_4_layers = [[training_x.shape[1], 1024, 256, 64, 32, 1],
                  [training_x.shape[1], 2048, 256, 64, 32, 1],
                  [training_x.shape[1], 1024, 128, 64, 32, 1],
                  [training_x.shape[1], 256, 64, 32, 16, 1],
                  [training_x.shape[1], 1024, 128, 64, 4, 1]]

models_4_layers_list = [Run_Model(nodes_4_layers[i]) for i in range(0, len(nodes_4_layers))]

In [ ]:
train_mse = []
val_mse = []
test_mse = []
for current_model in models_4_layers_list:

  current_model.compile(optimizer=tf.keras.optimizers.Adam(1e-4)
              ,loss="mean_squared_error"
              ,metrics=['mean_squared_error'])
  
  current_model.summary()

  history = current_model.fit(training_x, training_y ,
                      epochs = 200 ,validation_data=(val_x, val_y)
                      ,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_mean_squared_error', patience=2)]
                      )
  
  index = np.where(history.history.get("val_mean_squared_error") == np.min(history.history.get("val_mean_squared_error")))[0][0]
  train_mse.append(history.history.get("mean_squared_error")[index])
  val_mse.append(history.history.get("val_mean_squared_error")[index])
  test_mse.append(current_model.evaluate(test_x, test_y)[1])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              5096448   
                                                                 
 dense_1 (Dense)             (None, 256)               262400    
                                                                 
 dense_2 (Dense)             (None, 64)                16448     
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 1)                 33        
                                                                 
Total params: 5,377,409
Trainable params: 5,377,409
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
2779/2779 [=======================

In [ ]:
pd.DataFrame({"Nodes": nodes_4_layers, "Training": train_mse, "Validation": val_mse, "Test": test_mse})

ValueError: ignored